<a href="https://colab.research.google.com/github/MohammadAsad0/Medical-Diagnosis-Risk-Scoring-using-Bayesian-Networks/blob/main/PM_ML_Project_BN_Multi_Class_Heart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy pandas numpy scikit-learn ucimlrepo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, HillClimbSearch, BIC, K2, BayesianEstimator
from pgmpy.inference import VariableElimination

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Import for dataset
from ucimlrepo import fetch_ucirepo

from sklearn.model_selection import StratifiedKFold

In [ ]:
# Load Dataset
heart_disease = fetch_ucirepo(id=45)

X = heart_disease.data.features
y = heart_disease.data.targets

print("X shape: ", X.shape)
print("y shape: ", y.shape)

print(X.describe())
y.value_counts()

X shape:  (303, 13)
y shape:  (303, 1)
              age         sex          cp    trestbps        chol         fbs  \
count  303.000000  303.000000  303.000000  303.000000  303.000000  303.000000   
mean    54.438944    0.679868    3.158416  131.689769  246.693069    0.148515   
std      9.038662    0.467299    0.960126   17.599748   51.776918    0.356198   
min     29.000000    0.000000    1.000000   94.000000  126.000000    0.000000   
25%     48.000000    0.000000    3.000000  120.000000  211.000000    0.000000   
50%     56.000000    1.000000    3.000000  130.000000  241.000000    0.000000   
75%     61.000000    1.000000    4.000000  140.000000  275.000000    0.000000   
max     77.000000    1.000000    4.000000  200.000000  564.000000    1.000000   

          restecg     thalach       exang     oldpeak       slope          ca  \
count  303.000000  303.000000  303.000000  303.000000  303.000000  299.000000   
mean     0.990099  149.607261    0.326733    1.039604    1.600660    

,count
num,
0,164
1,55
2,36
3,35
4,13


In [ ]:
print("Null values in each Column: \n", X.isna().sum())

Null values in each Column: 
 age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
dtype: int64


# Preprocessing

In [ ]:
df = pd.concat([X, y], axis=1)

# Drop rows with missing values
df = df.dropna().reset_index(drop=True)

# Separate features and target
X = df.drop('num', axis=1)
y = df['num']

# Train / Test Split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

# Prepare training data
train_df = X_train.copy()
train_df['num'] = y_train.values

# Discretization

In [ ]:
# Bayesian Networks work best with discrete data
# We need to discretize continuous features

# Identify continuous and discrete features
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
discrete_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

print(f"\nContinuous features: {continuous_features}")
print(f"Discrete features: {discrete_features}")

disc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')

train_df[continuous_features] = disc.fit_transform(train_df[continuous_features])

# Convert to integer type (required for pgmpy)
for col in train_df.columns:
  train_df[col] = train_df[col].astype(int)

print("\nDiscretized training data sample:")
print(train_df.head())


Continuous features: ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
Discrete features: ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

Discretized training data sample:
     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
296    1    0   2         1     1    0        2        2      0        0   
82     0    1   3         2     2    0        2        2      0        0   
176    0    1   3         1     2    0        0        2      0        2   
246    2    1   2         1     0    1        2        0      0        0   
147    0    1   2         1     2    0        2        2      0        0   

     slope  ca  thal  num  
296      2   1     3    1  
82       1   0     3    0  
176      1   1     3    0  
246      1   0     3    0  
147      1   0     3    0  


# Structure Learning

In [ ]:
# Learn the Bayesian Network structure from data
# Using Hill Climb Search with BIC score
print("\nLearning network structure using Hill Climb Search...")

hc = HillClimbSearch(train_df)
best_model = hc.estimate(scoring_method=BIC(train_df))

print(f"\nLearned network edges:")
edges = best_model.edges()

for edge in edges:
  print(f"  {edge[0]} -> {edge[1]}")


Learning network structure using Hill Climb Search...


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned network edges:
  sex -> thal
  cp -> exang
  trestbps -> age
  chol -> sex
  fbs -> trestbps
  thalach -> slope
  exang -> thalach
  slope -> oldpeak
  thal -> num
  num -> ca
  num -> cp


In [ ]:
# Create Bayesian Network with learned structure
model = DiscreteBayesianNetwork(edges)

# Add domain knowledge: ensure 'target' is influenced by key medical features
# This encodes clinical knowledge into the structure
domain_edges = [
  ('cp', 'num'),      # Chest pain type is a strong predictor
  ('thalach', 'num'), # Maximum heart rate
  ('exang', 'num'),   # Exercise induced angina
  ('ca', 'num'),      # Number of major vessels
  ('thal', 'num'),    # Thalassemia
  ('sex', 'num'),     # Gender
  ('age', 'num'),     # Age
]

# Add domain knowledge edges if they don't exist
for edge in domain_edges:
  if edge not in model.edges() and not model.has_edge(edge[1], edge[0]):
    try:
      model.add_edge(edge[0], edge[1])
      print(f"Added domain knowledge edge: {edge[0]} -> {edge[1]}")
    except:
      pass

print(f"\nFinal network has {len(model.edges())} edges")
print(f"Network nodes: {sorted(model.nodes())}")

Added domain knowledge edge: sex -> num
Added domain knowledge edge: age -> num

Final network has 13 edges
Network nodes: ['age', 'ca', 'chol', 'cp', 'exang', 'fbs', 'num', 'oldpeak', 'sex', 'slope', 'thal', 'thalach', 'trestbps']


# Parameter Learning

In [ ]:
# Fit the model using Bayesian Estimation (with Dirichlet prior)
# This is more robust than Maximum Likelihood for small datasets
print("\nLearning conditional probability distributions...")

model.fit(train_df, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=10)

print("Model training complete!")
print(f"Number of CPDs learned: {len(model.get_cpds())}")

# Show one example CPD
print("\nExample CPD (Conditional Probability Distribution) for 'num':")
print(model.get_cpds('num'))


Learning conditional probability distributions...
Model training complete!
Number of CPDs learned: 13

Example CPD (Conditional Probability Distribution) for 'num':
+--------+----------------------+-----+----------------------+
| age    | age(0)               | ... | age(2)               |
+--------+----------------------+-----+----------------------+
| sex    | sex(0)               | ... | sex(1)               |
+--------+----------------------+-----+----------------------+
| thal   | thal(3)              | ... | thal(7)              |
+--------+----------------------+-----+----------------------+
| num(0) | 0.9783783783783785   | ... | 0.18548387096774194  |
+--------+----------------------+-----+----------------------+
| num(1) | 0.005405405405405406 | ... | 0.25806451612903225  |
+--------+----------------------+-----+----------------------+
| num(2) | 0.005405405405405406 | ... | 0.36693548387096775  |
+--------+----------------------+-----+----------------------+
| num(3) | 0.00

# Inference and Prediction

In [ ]:
# Create inference object
inference = VariableElimination(model)

# Prepare test data
test_data = X_test.copy()
test_data[continuous_features] = disc.transform(test_data[continuous_features])

# Convert to integer type
for col in test_data.columns:
  test_data[col] = test_data[col].astype(int)

print("\nMaking predictions on test set...")

# Get nodes that are actually in the model
model_nodes = set(model.nodes())
print(f"Nodes in the model: {model_nodes}")

# Make predictions
predictions = []
probabilities = []

for idx, row in test_data.iterrows():
  # Create evidence dictionary - only include features that are in the model
  evidence = {k: v for k, v in row.to_dict().items() if k in model_nodes and k != 'num'}

  # Query the probability distribution over target
  result = inference.query(variables=['num'], evidence=evidence)

  # Get the most likely class
  pred_class = result.values.argmax()
  predictions.append(pred_class)
  probabilities.append(result.values)

predictions = np.array(predictions)


Making predictions on test set...
Nodes in the model: {'thal', 'oldpeak', 'exang', 'trestbps', 'fbs', 'ca', 'num', 'sex', 'thalach', 'chol', 'slope', 'age', 'cp'}


# Evaluation

In [ ]:
# Calculate accuracy
bn_accuracy = accuracy_score(y_test, predictions)

print(f"\nTest Accuracy: {bn_accuracy:.4f} ({bn_accuracy*100:.2f}%)")

# Classification report
class_names = [f'Class {i}' for i in sorted(y.unique())]
print("\nClassification Report:")
print(classification_report(y_test, predictions, target_names=class_names, zero_division=0))

# Confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm,
                     index=[f'True {i}' for i in sorted(y.unique())],
                     columns=[f'Pred {i}' for i in sorted(y.unique())])
print(cm_df)

# Prediction distribution
print("\nPrediction distribution:")
pred_dist = pd.Series(predictions).value_counts().sort_index()
print(pred_dist)

print("\nActual test set distribution:")
print(y_test.value_counts().sort_index())


Test Accuracy: 0.6667 (66.67%)

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.86      0.94      0.90        32
     Class 1       0.30      0.27      0.29        11
     Class 2       0.50      0.43      0.46         7
     Class 3       0.50      0.57      0.53         7
     Class 4       0.00      0.00      0.00         3

    accuracy                           0.67        60
   macro avg       0.43      0.44      0.44        60
weighted avg       0.63      0.67      0.65        60


Confusion Matrix:
        Pred 0  Pred 1  Pred 2  Pred 3  Pred 4
True 0      30       2       0       0       0
True 1       4       3       1       2       1
True 2       1       1       3       2       0
True 3       0       2       1       4       0
True 4       0       2       1       0       0

Prediction distribution:
0    35
1    10
2     6
3     8
4     1
Name: count, dtype: int64

Actual test set distribution:
num
0    32
1    11
2     7
3  

# Cross-Validation

In [ ]:
# Perform manual cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

print("\nPerforming 5-fold cross-validation...")

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), 1):
  # Prepare fold data
  X_fold_train = X.iloc[train_idx].copy()
  y_fold_train = y.iloc[train_idx].copy()
  X_fold_val = X.iloc[val_idx].copy()
  y_fold_val = y.iloc[val_idx].copy()

  # Discretize
  fold_discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')

  fold_train_data = X_fold_train.copy()
  fold_train_data['num'] = y_fold_train.values
  fold_train_data[continuous_features] = fold_discretizer.fit_transform(fold_train_data[continuous_features])

  for col in fold_train_data.columns:
    fold_train_data[col] = fold_train_data[col].astype(int)

  # Learn structure and fit
  fold_hc = HillClimbSearch(fold_train_data)
  fold_best_model = fold_hc.estimate(scoring_method=BIC(fold_train_data))
  fold_model = DiscreteBayesianNetwork(fold_best_model.edges())

  # Add domain knowledge
  for edge in domain_edges:
    if edge not in fold_model.edges() and not fold_model.has_edge(edge[1], edge[0]):
      try:
        fold_model.add_edge(edge[0], edge[1])
      except:
        pass

  fold_model.fit(fold_train_data, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=10)

  # Predict
  fold_inference = VariableElimination(fold_model)
  fold_val_data = X_fold_val.copy()
  fold_val_data[continuous_features] = fold_discretizer.transform(fold_val_data[continuous_features])

  for col in fold_val_data.columns:
    fold_val_data[col] = fold_val_data[col].astype(int)

  fold_predictions = []
  fold_model_nodes = set(fold_model.nodes())

  for idx, row in fold_val_data.iterrows():
    # Only use features that are in the model
    evidence = {k: v for k, v in row.to_dict().items() if k in fold_model_nodes and k != 'num'}
    result = fold_inference.query(variables=['num'], evidence=evidence)
    fold_predictions.append(result.values.argmax())

  fold_accuracy = accuracy_score(y_fold_val, fold_predictions)
  cv_scores.append(fold_accuracy)
  print(f"Fold {fold}: {fold_accuracy:.4f}")

cv_scores = np.array(cv_scores)
print(f"\n5-Fold CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")



Performing 5-fold cross-validation...


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_discretization.py:306: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  warnings.warn(


  0%|          | 0/1000000 [00:00<?, ?it/s]

Fold 1: 0.6500


  0%|          | 0/1000000 [00:00<?, ?it/s]

...

Fold 3: 0.4746


  0%|          | 0/1000000 [00:00<?, ?it/s]

Fold 4: 0.5593


  0%|          | 0/1000000 [00:00<?, ?it/s]

Fold 5: 0.5593

5-Fold CV Accuracy: 0.5586 (+/- 0.0557)


# Final Results

In [ ]:
print(f"Test Accuracy: {bn_accuracy:.4f}")
print(f"CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
print(f"Number of edges in network: {len(model.edges())}")
print(f"Number of parameters (CPDs): {len(model.get_cpds())}")

Test Accuracy: 0.6167
CV Accuracy: 0.5586 (+/- 0.0557)
Number of edges in network: 10
Number of parameters (CPDs): 10
